# Todo
* think about storing the data and all the data.... sqllite, HDF5, pickle, csv
* better comments
* once through to clean up

In [1]:
import ssl
import re
import pandas as pd
import numpy as np

# gets rid of ssl errors
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

# set this to the list of expected columns
int64_columns = ['PKA', 'PKP', 'PKF', 'K4A', 'K4P', 'K4F', 'K5A', 'K5P', 'K5F',
                 '001', '002', '003', '004', '005', '006', '007', '008', '009',
                 '010', '011', '012', 'school_year']
string_columns = ['LEA Name', 'LEA Type', 'County', 'School Name']
expected_columns = string_columns + int64_columns

url = "https://www.education.pa.gov/DataAndReporting/Enrollment/Pages/PublicSchEnrReports.aspx"
host = re.sub(r'(https*://[^/]+).*','\\1',url)
req = Request(url)
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "html")

links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))

In [3]:
# this filters down the list to just what we are looking for
links = list( filter(lambda l: re.search(r'Enrollment%20Public%20Schools*%2020\d\d-\d\d\.xlsx*$', str(l) ) , links))

In [4]:
# this will put the host part of the URL back on the beginning of the link
links = list(map( lambda l: host + l, links))

In [5]:
# this will do the download, just pass it a sheet and URL or file name
def download_excel(file,sheet,skiprows):
        # download and read in the sheet
        df = pd.DataFrame()
        try:
            df = pd.read_excel(file,
                       sheet_name=sheet,
                       skiprows=skiprows)
        except:
            print( 'Error: not able to download: ' + file )
            
        return df

In [10]:
# builds one giant dataframe
df = pd.DataFrame()
count=0
for l in links: 
    if count < 120:
        # find the start year from the URL
        school_year = re.sub('^https.+(\d\d\d\d)-\d\d.xlsx*$','\\1',l)
        #print( school_year + ": " + l )

        # download and read in the sheet
        if int(school_year) >= 2011:
            df1 = download_excel(l,'LEA and School', 4)
        elif int(school_year) == 2010:
            df1 = download_excel(l,'School', 4)
        elif int(school_year) == 2009:
            df1 = download_excel(l,'School - Data File', 4)
        elif int(school_year) == 2008:
            df1 = download_excel(l,'School - Datafile', 4)
        elif int(school_year) == 2007:
            df1 = download_excel(l,'LEA - Data File', 6)
        elif int(school_year) == 2006:
            df1 = download_excel(l,'School Enrollments', 1)
        elif int(school_year) == 2005:
            df1 = download_excel(l,'School Enrollments by LEA', 2)
        else:     
            df1 = download_excel(l,'School Enrollments', 2)


        # 2006 has some bad columns in it. this makes them conform
        col = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
        for c in col:
            if c in df1.columns:
                nc = "00" + str(c)
                nc = nc[-3:]
                df1.rename(columns={c: nc}, inplace=True)
        
        # more 2006. They don't break K into AM, PM, Full. So put all numbers in AM
        if 'PreK' in df1.columns:
            df1.rename(columns={"PreK": "PKA"}, inplace=True)
        if 'K4' in df1.columns:
            df1.rename(columns={"K4": "K4A"}, inplace=True)
        if 'K5' in df1.columns:
            df1.rename(columns={"K5": "K5A"}, inplace=True)
        
        # 2006 uses county name instead of county
        if "County Name" in df1.columns:
            df1.rename(columns={"County Name": "County"}, inplace=True)
        
        # 2011 has a 7 as a column name. renaming that. its an int 7 not a string 7. 
        if 7 in df1.columns:
            df1.rename(columns={7: "007"}, inplace=True)
 
        # some of the older files have pivot table looking reports. Where say the 
        # county is in row 1 with 200 schools, and the next 199 rows they don't
        # repeat the county. So this fills in County and the School District or 
        # LEA Name
        fields = ['County', 'LEA Name']
        for f in fields:
            field = ''
            for i, row in df1.iterrows():
                if not pd.isna(df1.at[i,f]):
                    field = df1.at[i,f]
                else:
                    df1.at[i,f] = field

        # add LEA Type by using the end of the LEA Name. For example Avon Grove School
        # district would be "Avon Grove SD". So pulling off the right most string without 
        # a space will get me the LEA Type
        if 'LEA Type' not in df1.columns:
            pd.Series(df1['LEA Name'], dtype="string") # doing this now because in 2005 its a strange data type
            df1['LEA Type'] = df['LEA Name'].apply(lambda x: re.sub(' (\S+)$','\\1',x))       

        # add in school year
        df1['school_year'] = school_year
         
        # get rid of any row where school number isn't a number
        if 'School Number' in df1.columns:
            df1['School Number'] = pd.Series(df1['School Number'], dtype="string")
            df1 = df1[df1['School Number'].str.contains('^[\d\.]+$', regex= True, na=False)]
        
        # fix datatypes
        for col in int64_columns:
            # if doesn't exist create it
            if col not in df1.columns:
                df1[col] = np.nan
            df1[col] = pd.Series(df1[col], dtype="Int64")   
                
        for col in string_columns:
            # if doesn't exist create it
            if col not in df1.columns:
                df1[col] = np.nan
            df1[col] = pd.Series(df1[col], dtype="string") 

        # this drops un-used columns. compares retrieved col to expected ones
        for col in df1.columns.difference(expected_columns):
            if col in df1.columns:
                df1.drop(columns=[col], inplace=True)

        # drops any rows where LEA Name ends in Total
        df1 = df1[~df1['LEA Name'].str.contains('^.*Total$', regex= True, na=False)]
                
        # concat the newly downloaded df onto the larger one
        df = pd.concat([df, df1],
                       ignore_index=True,)
    count = count + 1

# convert header names to strings
df.columns = df.columns.map(str)

In [11]:
df.school_year.unique()

array(['2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013',
       '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005',
       '2004'], dtype=object)

In [8]:
# save the data out for another script to consume
df.to_pickle("../data/basic_enrollment.pkl.bz2", compression='infer')

In [9]:
#df.to_excel("../data/basic_enrollment.xlsx")